In [ ]:
import numpy as np
from sklearn import metrics
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred - y_true) / y_true))

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns 
import matplotlib.pyplot as plt 
from colorama import Fore
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, mean_squared_error
import math
from sklearn.preprocessing import LabelEncoder
import warnings # Supress warnings 
warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
np.random.seed(7)

# 数据导入

In [ ]:
#df = label
df = pd.read_csv(r"附件1-区域15分钟负荷数据.csv")
df  = df.rename(columns={'数据时间':'date'})
# df = df[126000:]  #选取多少数据进行预测

In [ ]:
#划分数据集
train_size = int(0.85 * len(df))
test_size = len(df) - train_size
df = df.fillna(0)
univariate_df = df[['date', '总有功功率（kw）']].copy()
univariate_df.columns = ['ds', 'y']

train = univariate_df.iloc[:train_size, :]

x_train, y_train = pd.DataFrame(univariate_df.iloc[:train_size, 0]), pd.DataFrame(univariate_df.iloc[:train_size, 1])
x_valid, y_valid = pd.DataFrame(univariate_df.iloc[train_size:, 0]), pd.DataFrame(univariate_df.iloc[train_size:, 1])

print(len(train), len(x_valid))

In [ ]:
# 模型预测+精度分析
trend_evaluate = sm.tsa.arma_order_select_ic(y_train, ic=['aic', 'bic'], trend='nc')
print('train AIC', trend_evaluate.aic_min_order)
print('train BIC', trend_evaluate.bic_min_order)

In [ ]:
# Fit model
model = ARIMA(y_train, order=(4,0,0))
model_fit = model.fit()

# Prediction with ARIMA
y_pred = model_fit.forecast(len(y_valid))
# print(y_pred)
# Calcuate metrics
score_mae = mean_absolute_error(y_valid, y_pred[0])
score_rmse = math.sqrt(mean_squared_error(y_valid, y_pred[0]))

y_true = np.array(y_valid)
y_pred = np.array(y_pred[0])
print('MSE:',metrics.mean_squared_error(y_true, y_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_true, y_pred)))
print('MAE:',metrics.mean_absolute_error(y_true, y_pred)) 
print('MAPE:',mape(y_true, y_pred))

In [ ]:
# Fit model
model = ARIMA(y_train, order=(4,0,0))
model_fit = model.fit()

# Prediction with ARIMA
y_pred = model_fit.forecast(len(y_valid))
# Calcuate metrics
score_mae = mean_absolute_error(y_valid, y_pred[1])
score_rmse = math.sqrt(mean_squared_error(y_valid, y_pred[1]))

y_true = np.array(y_valid)
y_pred = np.array(y_pred[1])
print('MSE:',metrics.mean_squared_error(y_true, y_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_true, y_pred)))
print('MAE:',metrics.mean_absolute_error(y_true, y_pred)) 
print('MAPE:',mape(y_true, y_pred))

In [ ]:
y = pd.read_excel(r'未来三个月.xls')
pred =  model_fit.forecast(len(y))

In [ ]:
# pd.DataFrame([pred[0],pred[1],pred[2][:,0],pred[2][:,1]]).T

In [ ]:
pred =  model_fit.forecast(len(y))
pred = pd.DataFrame(pred[0])
pred.index = y.index
y['数据时间'] = y['数据时间'].astype(str)
y['总有功功率(KM)'] = pred

In [ ]:
y['数据时间'] = y['数据时间'].apply(lambda x: x.split(' ')[0])

In [ ]:
y_max = y['总有功功率(KM)'].max()
y[y['总有功功率(KM)']==y_max]

In [ ]:
y_min = y['总有功功率(KM)'].min()
y[y['总有功功率(KM)']== y_min]

In [ ]:
pd.DataFrame(y).to_csv('未来三个月预测.csv',index=0)

In [ ]:
# y_min = y.groupby(['数据时间'])['总有功功率(KM)'].min()
# pd.DataFrame(y_max).to_csv('未来三个月每天最小值.csv')